# Import

In [389]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 시각화시 한글 깨짐 방지
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')  # Linux 사용자의 경우
# 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

from summarytools import dfSummary
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

# Data Load

In [390]:
train = pd.read_csv('../../data/raw/train.csv').drop(columns=['ID'])
test = pd.read_csv('../../data/raw/test.csv').drop(columns=['ID'])

# Data Pre-processing

### [도은_논문기반] 결측이 많은 행 위주의 다운 샘플링

In [118]:
sum(train['임신 성공 여부'] == 1) 

66228

In [113]:
train_success = train.loc[train['임신 성공 여부'] == 1]
train_fail = train.loc[train['임신 성공 여부'] == 0]
display(train_fail.isna().sum(axis = 1).value_counts())


cnt_success = sum(train['임신 성공 여부'] == 1) 
train_dwnsmp1 = train_fail.loc[train_fail.isna().sum(axis = 1) < 6] # 결측값이 있는 열이 6개보다 작으면 무조건 포함
train_dwnsmp2 = train_fail[(train_fail.isna().sum(axis = 1) == 6)] # 결측값이 있는 열이 6개인 행에서 임신성공 행 수에 맞추어 랜덤샘플
for_sample_cnt = cnt_success - len(train_dwnsmp1)
print(for_sample_cnt)


import random
idx = random.sample(range(0, len(train_dwnsmp2)), for_sample_cnt)
train_dwnsmp = pd.concat([train_success, train_dwnsmp1,train_dwnsmp2.iloc[idx,:]])


train = train_dwnsmp 
train.info()

6     115135
7      53516
8      10611
30      5228
5       4695
4        650
29       252
3         36
dtype: int64

60847
<class 'pandas.core.frame.DataFrame'>
Int64Index: 132456 entries, 13 to 78449
Data columns (total 68 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   시술 시기 코드               132456 non-null  object 
 1   시술 당시 나이               132456 non-null  object 
 2   임신 시도 또는 마지막 임신 경과 연수  6782 non-null    float64
 3   시술 유형                  132456 non-null  object 
 4   특정 시술 유형               132456 non-null  object 
 5   배란 자극 여부               132456 non-null  int64  
 6   배란 유도 유형               132456 non-null  object 
 7   단일 배아 이식 여부            131645 non-null  float64
 8   착상 전 유전 검사 사용 여부       2334 non-null    float64
 9   착상 전 유전 진단 사용 여부       131645 non-null  float64
 10  남성 주 불임 원인             132456 non-null  int64  
 11  남성 부 불임 원인             132456 non-null  int64  
 12  여성 주 불임 원인             132456 non-null  int64  
 13  여성 부 불임 원인             132456 non-null  int64  
 14  부부 주 불임 원인             132456 

### [도은_논문기반] 25개의 지침을 받은 변수만으로 해보기.

In [167]:
paper_column = ['시술 당시 나이', 
                'IVF 시술 횟수',
                'IVF 임신 횟수',
                'IVF 출산 횟수',
                '여성 주 불임 원인',
                '여성 부 불임 원인',
                '남성 주 불임 원인',
                '남성 부 불임 원인',
                '부부 주 불임 원인',
                '부부 부 불임 원인',
                '불임 원인 - 난관 질환',
                '불임 원인 - 배란 장애',
                '불임 원인 - 남성 요인',
                '불명확 불임 원인',
                '불임 원인 - 자궁내막증',
                '불임 원인 - 자궁경부 문제',
                '불임 원인 - 여성 요인',
                '불임 원인 - 정자 농도',
                '불임 원인 - 정자 형태',
                '불임 원인 - 정자 운동성',
                '불임 원인 - 정자 면역학적 요인',
                '배란 자극 여부',
                '난자 출처',
                '정자 출처',
                '신선 배아 사용 여부',
                '동결 배아 사용 여부',
                '해동 난자 수',
                '저장된 배아 수',
                '파트너 정자와 혼합된 난자 수',
                '이식된 배아 수']
len(paper_column)

# 논문에서는 '난자 출처', '정자 출처', '불임 원인 - 정자 면역학적 요인','배란 자극 여부','불임 원인 - 여성 요인' -> 분산이 0 이어서 제외

30

In [168]:
train = train[paper_column+['임신 성공 여부']]
test = test[paper_column]

### try [배아 사용 조합에 따른 배아 이식 경과일 평균으로 결측 처리]
- public 0.7407619508 -> 0.7409045012

In [391]:
try:
    display(dfSummary(train[['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', "배아 이식 경과일"]]))
except :
    display(dfSummary(train[['동결 배아 사용 여부', '신선 배아 사용 여부']]))

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,동결 배아 사용 여부[float64],1. 0.02. 1.03. nan,"209,934 (81.9%)40,126 (15.7%)6,291 (2.5%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABjklEQVR4nO3bQWrCUBiF0T9FSjtJEMHFuIgutovIbkR8g7Z0kg46qdjpuwl4zgZewE9DJHdYlqUg4WntC+BxDFX1WlXPa18Iq/heluUzddjucDi8jeO4Tx3IdrTWLsMwvKeC243juD+dTp/TNH0lDmQbrtfryzzP+/P5/FxVmdiqqqZp+joejx+JA9mU1+RhHhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEbOr+n23ae0LIWuNz3zXWrvM87yv8LtNrK+1dqmq79R5NgiPLbpBGEz5SEn8skW/PWxX93VVesHDdnVdV62x4GG7EusqT7lUlT91CRIbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHTdV1ltcVf3ddV6QUP22WDQIx1FTEeEIj57zbqtkcXd1M+0zt6uZnymd7R039TPtM7uvCAQIzYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI2Ymymf6R093U35TO/oxQaBGFM+Yn4ASryjShr/3MkAAAAASUVORK5CYII="">","6,291(2.5%)"
2,신선 배아 사용 여부[float64],1. 1.02. 0.03. nan,"210,136 (82.0%)39,924 (15.6%)6,291 (2.5%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABjklEQVR4nO3bQWrCUBiF0T9FSjtJEMHFuIgutovIbkR8g7Z0kg46qdjpuwl4zgZewE9DJHdYlqUg4WntC+BxDFX1WlXPa18Iq/heluUzddjucDi8jeO4Tx3IdrTWLsMwvKeC243juD+dTp/TNH0lDmQbrtfryzzP+/P5/FxVmdiqqqZp+joejx+JA9mU1+RhHhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEbOr+n23ae0LIWuNz3zXWrvM87yv8LtNrK+1dqmq79R5NgiPLbpBGEz5SEn8skW/PWxX93VVesHDdnVdV62x4GG7EusqT7lUlT91CRIbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHTdV1ltcVf3ddV6QUP22WDQIx1FTEeEIj57zbqtkcXd1M+0zt6uZnymd7R039TPtM7uvCAQIzYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI2Ymymf6R093U35TO/oxQaBGFM+Yn4ASryjShr/3MkAAAAASUVORK5CYII="">","6,291(2.5%)"
3,기증 배아 사용 여부[float64],1. 0.02. nan3. 1.0,"247,602 (96.6%)6,291 (2.5%)2,458 (1.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABmUlEQVR4nO3cMWrbcBjG4U9BhGaREAIfxofIYXsI3cYYa0hCFnVoSp3aWYr1CpznGTX9hx+WZfS6WZalIOFh6wPwfTRV9VRVj1sfhLvzvizL6/mFdhzH567rhq1OxH2a5/nYNM3P8+DaruuG/X7/2vf925aH436cTqcf0zQNh8Phsar+xlZV1ff92263e9nsdNyjp38veEAgRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERkxb9fv9o60Pwv34qqd2nufjNE1DXXn/CP7XPM/Hqno/v2aDwFouNgiNKR8p1z7ZLoqEW7hYV11bxcAtfFpXfbWKgVu4tq7yVMoq/KhLjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjZhP6yorK9Z0sa66toqBW7BBIMa6ihgPCMT8uY2WWydra8dxfK6qMt9jbQ9d1w0fu1F/wcCqfGcjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMe3HdK/KfI+V2SAQY8pHzC8Q0oNsdhqPbAAAAABJRU5ErkJggg=="">","6,291(2.5%)"
4,배아 이식 경과일[float64],1. 5.02. 3.03. nan4. 2.05. 0.06. 1.07. 4.08. 6.09. 7.0,"81,459 (31.8%)57,924 (22.6%)43,566 (17.0%)35,078 (13.7%)24,904 (9.7%)6,053 (2.4%)4,504 (1.8%)2,773 (1.1%)90 (0.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAADPCAYAAADifqNtAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADyElEQVR4nO3dMW7bWBiF0Z+EkIkaCoIBL2HSeQFaRBabRWgBLmcPhmEWo0wapUhmYI8dO5L8Lmn5nC7dKz48gQSv0+33+4KEfuoD8H6IjRixESM2YsRGjNiIERsxXVUtq+rD1Ac50bf9fr+b+hA8b3FxcfF5GIb11Ac5xTiOt13XfRHcvC2GYVhvNpvdarX6OvVhjnF3d/dxu92ub25uPlSV2GZsUVW1Wq2+Xl5e/j31YU6wnPoAvMwDAjFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNGLERs6j68ZnO1Ac51ls++3uzGMfxdrvdrusNf6YzjuNtVX2b+hw8z2fhxHT+1gcpc7vZ3FBnbFaDF8OV8zabwYvhyvmb2+DlzT4R8zIvdYkRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNmNkMXuZwBtqa1eDFcOW8+SycGIMXYjwgEJP8GfUT+c7F1lWWU0TWVZZTVGXXVZO/WmFaHhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjZjIuspyiqrguspyCp+FE2NdRUyrm80txiNNBi/GLTzl1Qcvxi38SqvBi3ELj3ipS4zYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEvPrgxbiFX2kyeDFu4Sk+CyfG4IWYU282Nxi/7aTBi2ELhzh68GLYwqFOHbwYtvDbvNQlRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERszR6yorKg510rrKiopD2CAQY11FzFM3m9uKJh6tqyymaOXBuspiipaeWldZTNGEl7rEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERsyDwYsRCy09GrwYsdCKz8KJMXgh5v83m1uNZh4M

In [392]:
def fill_missing_embryo_days(train_df, test_df):
    """
    '시술 유형'이 'IVF'인 경우에만 배아 사용 조합에 따른 배아 이식 경과일 결측값을 채우는 함수
    """

    # IVF 조건 필터링
    train_ivf = train_df[train_df["시술 유형"] == "IVF"].copy()
    test_ivf = test_df[test_df["시술 유형"] == "IVF"].copy()

    # 배아 사용 조합 컬럼 생성
    train_ivf["배아 사용 조합"] = train_ivf[['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']].astype(str).agg('-'.join, axis=1)
    test_ivf["배아 사용 조합"] = test_ivf[['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']].astype(str).agg('-'.join, axis=1)

    # 학습 데이터에서 조합별 평균값 계산
    mean_days_by_group = train_ivf.groupby("배아 사용 조합")["배아 이식 경과일"].mean()
    

    # 학습 데이터 결측값 채우기
    train_ivf["배아 이식 경과일"] = train_ivf.apply(
        lambda row: mean_days_by_group[row["배아 사용 조합"]] if pd.isna(row["배아 이식 경과일"]) else row["배아 이식 경과일"],
        axis=1
    )

    # 테스트 데이터 결측값 채우기 (train에서 계산된 평균 사용)
    test_ivf["배아 이식 경과일"] = test_ivf.apply(
        lambda row: mean_days_by_group.get(row["배아 사용 조합"], row["배아 이식 경과일"]) if pd.isna(row["배아 이식 경과일"]) else row["배아 이식 경과일"],
        axis=1
    )

    # 원본 데이터에 반영
    train_df.loc[train_df["시술 유형"] == "IVF", "배아 이식 경과일"] = train_ivf["배아 이식 경과일"]
    test_df.loc[test_df["시술 유형"] == "IVF", "배아 이식 경과일"] = test_ivf["배아 이식 경과일"]
    
    print(f"배아 사용 조합에 따른 IVF 배아 이식 경과일의 평균 : \n{mean_days_by_group}")

    return train_df, test_df


def fill_missing_embryo_days_paper(train_df, test_df):
    """
    '시술 유형'이 'IVF'인 경우에만 배아 사용 조합에 따른 배아 이식 경과일 결측값을 채우는 함수
    """

    # # IVF 조건 필터링
    # train_ivf = train_df[train_df["시술 유형"] == "IVF"].copy()
    # test_ivf = test_df[test_df["시술 유형"] == "IVF"].copy()
    train_ivf = train
    test_ivf = test

    # 배아 사용 조합 컬럼 생성
    train_ivf["배아 사용 조합"] = train_ivf[['동결 배아 사용 여부', '신선 배아 사용 여부']].astype(str).agg('-'.join, axis=1)
    test_ivf["배아 사용 조합"] = test_ivf[['동결 배아 사용 여부', '신선 배아 사용 여부']].astype(str).agg('-'.join, axis=1)

    # # 학습 데이터에서 조합별 평균값 계산
    # mean_days_by_group = train_ivf.groupby("배아 사용 조합")["배아 이식 경과일"].mean()
    

    # # 학습 데이터 결측값 채우기
    # train_ivf["배아 이식 경과일"] = train_ivf.apply(
    #     lambda row: mean_days_by_group[row["배아 사용 조합"]] if pd.isna(row["배아 이식 경과일"]) else row["배아 이식 경과일"],
    #     axis=1
    # )

    # # 테스트 데이터 결측값 채우기 (train에서 계산된 평균 사용)
    # test_ivf["배아 이식 경과일"] = test_ivf.apply(
    #     lambda row: mean_days_by_group.get(row["배아 사용 조합"], row["배아 이식 경과일"]) if pd.isna(row["배아 이식 경과일"]) else row["배아 이식 경과일"],
    #     axis=1
    # )

    # # 원본 데이터에 반영
    # train_df.loc[train_df["시술 유형"] == "IVF", "배아 이식 경과일"] = train_ivf["배아 이식 경과일"]
    # test_df.loc[test_df["시술 유형"] == "IVF", "배아 이식 경과일"] = test_ivf["배아 이식 경과일"]
    
    # print(f"배아 사용 조합에 따른 IVF 배아 이식 경과일의 평균 : \n{mean_days_by_group}")

    return train_df, test_df


try:
    train, test = fill_missing_embryo_days(train, test)
except:
    train, test = fill_missing_embryo_days_paper(train, test)

배아 사용 조합에 따른 IVF 배아 이식 경과일의 평균 : 
배아 사용 조합
0.0-0.0-0.0    5.000000
0.0-1.0-0.0    3.794576
0.0-1.0-1.0    4.119822
1.0-0.0-0.0    0.665287
1.0-0.0-1.0    0.332629
1.0-1.0-0.0    3.053922
Name: 배아 이식 경과일, dtype: float64


In [393]:
try:
    display(dfSummary(train[['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', "배아 이식 경과일"]]))
except:
    display(dfSummary(train[['동결 배아 사용 여부', '신선 배아 사용 여부']]))

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,동결 배아 사용 여부[float64],1. 0.02. 1.03. nan,"209,934 (81.9%)40,126 (15.7%)6,291 (2.5%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABjklEQVR4nO3bQWrCUBiF0T9FSjtJEMHFuIgutovIbkR8g7Z0kg46qdjpuwl4zgZewE9DJHdYlqUg4WntC+BxDFX1WlXPa18Iq/heluUzddjucDi8jeO4Tx3IdrTWLsMwvKeC243juD+dTp/TNH0lDmQbrtfryzzP+/P5/FxVmdiqqqZp+joejx+JA9mU1+RhHhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEbOr+n23ae0LIWuNz3zXWrvM87yv8LtNrK+1dqmq79R5NgiPLbpBGEz5SEn8skW/PWxX93VVesHDdnVdV62x4GG7EusqT7lUlT91CRIbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHTdV1ltcVf3ddV6QUP22WDQIx1FTEeEIj57zbqtkcXd1M+0zt6uZnymd7R039TPtM7uvCAQIzYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI2Ymymf6R093U35TO/oxQaBGFM+Yn4ASryjShr/3MkAAAAASUVORK5CYII="">","6,291(2.5%)"
2,신선 배아 사용 여부[float64],1. 1.02. 0.03. nan,"210,136 (82.0%)39,924 (15.6%)6,291 (2.5%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABjklEQVR4nO3bQWrCUBiF0T9FSjtJEMHFuIgutovIbkR8g7Z0kg46qdjpuwl4zgZewE9DJHdYlqUg4WntC+BxDFX1WlXPa18Iq/heluUzddjucDi8jeO4Tx3IdrTWLsMwvKeC243juD+dTp/TNH0lDmQbrtfryzzP+/P5/FxVmdiqqqZp+joejx+JA9mU1+RhHhCIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjRixEbOr+n23ae0LIWuNz3zXWrvM87yv8LtNrK+1dqmq79R5NgiPLbpBGEz5SEn8skW/PWxX93VVesHDdnVdV62x4GG7EusqT7lUlT91CRIbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHTdV1ltcVf3ddV6QUP22WDQIx1FTEeEIj57zbqtkcXd1M+0zt6uZnymd7R039TPtM7uvCAQIzYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI2Ymymf6R093U35TO/oxQaBGFM+Yn4ASryjShr/3MkAAAAASUVORK5CYII="">","6,291(2.5%)"
3,기증 배아 사용 여부[float64],1. 0.02. nan3. 1.0,"247,602 (96.6%)6,291 (2.5%)2,458 (1.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABmUlEQVR4nO3cMWrbcBjG4U9BhGaREAIfxofIYXsI3cYYa0hCFnVoSp3aWYr1CpznGTX9hx+WZfS6WZalIOFh6wPwfTRV9VRVj1sfhLvzvizL6/mFdhzH567rhq1OxH2a5/nYNM3P8+DaruuG/X7/2vf925aH436cTqcf0zQNh8Phsar+xlZV1ff92263e9nsdNyjp38veEAgRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERkxb9fv9o60Pwv34qqd2nufjNE1DXXn/CP7XPM/Hqno/v2aDwFouNgiNKR8p1z7ZLoqEW7hYV11bxcAtfFpXfbWKgVu4tq7yVMoq/KhLjNiIERsxYiNGbMSIjRixESM2YsRGjNiIERsxYiNGbMSIjZhP6yorK9Z0sa66toqBW7BBIMa6ihgPCMT8uY2WWydra8dxfK6qMt9jbQ9d1w0fu1F/wcCqfGcjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMe3HdK/KfI+V2SAQY8pHzC8Q0oNsdhqPbAAAAABJRU5ErkJggg=="">","6,291(2.5%)"
4,배아 이식 경과일[float64],Mean (sd) : 3.3 (1.6)min < med < max:0.0 < 3.0 < 7.0IQR (CV) : 3.0 (2.0),13 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuNSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/xnp5ZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACeUlEQVR4nO3cv27aUBiG8e+UPzFQGRAtbKwMlTowchG92F5CB1YW7oANySAjXAhg3KFLBkhisMGv9fz2E51ITwgHvmOXJIkBRffl2RsAPqP67A0gW865hpnVb1x+SJJkl+V+skKoJeKca/R6vV++73dvWb/ZbNbOud9FjJVQy6Xu+353Mpns2u32Ps3CMAy96XTaDYKgbmaEivy12+19v9//e8PSRuabyQiHKUggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEggVEhgcPqKst49UkWoF5T57pEqQr2stHePVBHqO8p490gVhylIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRIIFRI4Lv+HMRxXDMz3zl3y3JGBC8g1IxFUVQ7Ho8/h8NhpVKppJq8MmNE8BpCzdjhcKh6ntccj8f7wWCwTrOWEcHrCh2q8pR9q9ViRDBDhQ2VKXu8VdhQjSl7vFHkUM2MKXv8x+eokECokPDhv37lkzfK491QOXmjKD56ReXkjUL41KmfkzeejcMUJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJBAqJOR6ue/OR9v45/O5lvGWICq3UO99tM3xePTiOP6x3+//mNkts7AokdxCvefRNmZmi8WiO5/Pm6fTqZLH/qAl93v9tz7aZr1eczvgwe58q2aW42XOwj+AAo9x71s1s3wvc34q1DAMvbQ/eLvdemZmURR5y+Wy+cj1YRh6cRxX7b6DXPXRv/cz971arb56ntccjUau0+m8pl2/2+1eZrPZ9yAIvjnnNmnXm9lrkiRX/0D+ARTxKalXLqnWAAAAAElFTkSuQmCC"">","6,291(2.5%)"


### 결측 처리
- '알 수 없음', 'Unknown','기록되지 않은 시행'은 non으로 처리

In [394]:
def remove_fully_missing_columns(train_df, test_df):
    """
    모든 관측치가 결측값(NaN)인 변수를 삭제하는 함수

    Parameters:
    - train_df: 학습 데이터 (DataFrame)
    - test_df: 테스트 데이터 (DataFrame)

    Returns:
    - train_df: 결측 변수 삭제 후 학습 데이터 (DataFrame)
    - test_df: 결측 변수 삭제 후 테스트 데이터 (DataFrame)
    """

    # 모든 값이 NaN인 컬럼 찾기
    train_missing_cols = train_df.columns[train_df.isna().all()].tolist()
    test_missing_cols = test_df.columns[test_df.isna().all()].tolist()

    # 삭제할 컬럼 리스트 생성
    cols_to_drop = list(set(train_missing_cols + test_missing_cols))

    # 데이터에서 해당 컬럼 삭제
    train_df = train_df.drop(columns=cols_to_drop)
    test_df = test_df.drop(columns=cols_to_drop)

    # 삭제된 컬럼 출력
    print(f"Train에서 삭제된 컬럼: {train_missing_cols}")
    print(f"Test에서 삭제된 컬럼: {test_missing_cols}")

    return train_df, test_df


train, test = remove_fully_missing_columns(train, test)


Train에서 삭제된 컬럼: []
Test에서 삭제된 컬럼: []


### 결측치 처리
- 삭제할 변수
    - "임신 시도 또는 마지막 임신 경과 연수", "배란 유도 유형", 
- 0으로 대체
    - "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부", "총 생성 배아 수", "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수",
    "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수", 
    "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부", "난자 해동 경과일", 
    "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"
- 1로 대체
    - "난자 채취 경과일"
- "NaN"으로 대체
    - "난자 출처"
- 이후에 따로 처리할 필요가 있는 변수
    - "시술 당시 나이", "특정 시술 유형", "배아 생성 주요 이유", "난자 출처", "정자 출처", "난자 기증자 나이", "정자 기증자 나이"

In [395]:
def imputation(train_df, test_df):
    """
    결측값 처리 함수
    
    - 삭제할 변수: "임신 시도 또는 마지막 임신 경과 연수", "배란 유도 유형"
    - 0으로 대체할 변수: (리스트 참고)
    - 1로 대체할 변수: "난자 채취 경과일"
    """

    # 삭제할 변수 목록
    cols_to_drop = ["임신 시도 또는 마지막 임신 경과 연수", "배란 유도 유형"]
    
    # 0으로 대체할 변수 목록
    cols_to_fill0 = [
        "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부", "총 생성 배아 수", "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수",
        "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부", "난자 해동 경과일",
        "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"
    ]
    
    # 1로 대체할 변수 목록
    cols_to_fill1 = ["난자 채취 경과일"]
    
    # "NAN"으로 대체할 변수 목록
    cols_to_fill_nan = ["난자 출처"]

    # 실제 존재하는 컬럼만 필터링
    drop_cols_existing = [col for col in cols_to_drop if col in train_df.columns]
    fill0_cols_existing = [col for col in cols_to_fill0 if col in train_df.columns]
    fill1_cols_existing = [col for col in cols_to_fill1 if col in train_df.columns]
    fill_nan_cols_existing = [col for col in cols_to_fill_nan if col in train_df.columns]

    # 데이터에서 해당 컬럼 삭제
    train_df = train_df.drop(columns=drop_cols_existing, errors='ignore')
    test_df = test_df.drop(columns=drop_cols_existing, errors='ignore')
    
    # 결측값 0으로 대체
    train_df[fill0_cols_existing] = train_df[fill0_cols_existing].fillna(0)
    test_df[fill0_cols_existing] = test_df[fill0_cols_existing].fillna(0)

    # 결측값 1로 대체
    train_df[fill1_cols_existing] = train_df[fill1_cols_existing].fillna(1)
    test_df[fill1_cols_existing] = test_df[fill1_cols_existing].fillna(1)
    
    # 결측값 NaN로 대체
    train_df[fill_nan_cols_existing] = train_df[fill_nan_cols_existing].fillna("NaN")
    test_df[fill_nan_cols_existing] = test_df[fill_nan_cols_existing].fillna("NaN")
    
    print(f"삭제된 변수 : {drop_cols_existing}")
    print(f"0으로 결측 대체 : {fill0_cols_existing}")
    print(f"1로 결측 대체 : {fill0_cols_existing}")
    print(f"NaN으로 결측 대체 : {fill_nan_cols_existing}")

    return train_df, test_df


def imputation_paper(train_df, test_df):
    """
    결측값 처리 함수
    
    - 삭제할 변수: "임신 시도 또는 마지막 임신 경과 연수", "배란 유도 유형"
    - 0으로 대체할 변수: (리스트 참고)
    - 1로 대체할 변수: "난자 채취 경과일"
    """

    # # 삭제할 변수 목록
    # cols_to_drop = ["임신 시도 또는 마지막 임신 경과 연수", "배란 유도 유형"]
    
    # 0으로 대체할 변수 목록
    cols_to_fill0 = [
        "단일 배아 이식 여부", "착상 전 유전 검사 사용 여부", "착상 전 유전 진단 사용 여부", "총 생성 배아 수", "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수",
        "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부", "PGD 시술 여부", "PGS 시술 여부", "난자 해동 경과일",
        "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일"
    ]
    
    # 1로 대체할 변수 목록
    cols_to_fill1 = ["난자 채취 경과일"]
    
    # "NAN"으로 대체할 변수 목록
    cols_to_fill_nan = ["난자 출처"]

    # 실제 존재하는 컬럼만 필터링
    drop_cols_existing = [col for col in cols_to_drop if col in train_df.columns]
    fill0_cols_existing = [col for col in cols_to_fill0 if col in train_df.columns]
    fill1_cols_existing = [col for col in cols_to_fill1 if col in train_df.columns]
    fill_nan_cols_existing = [col for col in cols_to_fill_nan if col in train_df.columns]

    # 데이터에서 해당 컬럼 삭제
    train_df = train_df.drop(columns=drop_cols_existing, errors='ignore')
    test_df = test_df.drop(columns=drop_cols_existing, errors='ignore')
    
    # 결측값 0으로 대체
    train_df[fill0_cols_existing] = train_df[fill0_cols_existing].fillna(0)
    test_df[fill0_cols_existing] = test_df[fill0_cols_existing].fillna(0)

    # 결측값 1로 대체
    train_df[fill1_cols_existing] = train_df[fill1_cols_existing].fillna(1)
    test_df[fill1_cols_existing] = test_df[fill1_cols_existing].fillna(1)
    
    # 결측값 NaN로 대체
    train_df[fill_nan_cols_existing] = train_df[fill_nan_cols_existing].fillna("NaN")
    test_df[fill_nan_cols_existing] = test_df[fill_nan_cols_existing].fillna("NaN")
    
    print(f"삭제된 변수 : {drop_cols_existing}")
    print(f"0으로 결측 대체 : {fill0_cols_existing}")
    print(f"1로 결측 대체 : {fill0_cols_existing}")
    print(f"NaN으로 결측 대체 : {fill_nan_cols_existing}")

    return train_df, test_df

train, test = imputation(train, test)

삭제된 변수 : ['임신 시도 또는 마지막 임신 경과 연수', '배란 유도 유형']
0으로 결측 대체 : ['단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']
1로 결측 대체 : ['단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']
NaN으로 결측 대체 : ['난자 출처']


### "특정 시술 유형'
- "Unknown" => nan으로 변경
- Unknown이 : 으로 묶인 경우 => Unknown 칼럼 생성
- "FER", "GIFT" 는Unknown으로

In [396]:
def encode_treatment_types(train_df, test_df):
    # 데이터 복사
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    train_df["특정 시술 유형"] = train_df['특정 시술 유형'].replace("FER", "Unknown").replace("GIFT", "Unknown")
    test_df["특정 시술 유형"] = test_df['특정 시술 유형'].replace("FER", "Unknown").replace("GIFT", "Unknown")

    # 특정 시술 유형에서 고유한 시술 종류 추출
    all_treatments = set()

    for df in [train_df, test_df]:
        if '특정 시술 유형' in df.columns:
            df['특정 시술 유형'] = df['특정 시술 유형'].astype(str).fillna("Unknown")  # 결측값 방지
            df['특정 시술 유형'].str.replace(" ", "").str.split(":|/").apply(all_treatments.update)
    
    # 'nan' 값이 존재하면 제거
    all_treatments.discard('nan')
    
    # 새로운 시술 유형 컬럼 생성 (초기값 0)
    for treatment in all_treatments:
        train_df[treatment] = 0
        test_df[treatment] = 0

    # 해당 시술이 포함된 경우 1로 설정
    for df in [train_df, test_df]:
        if '특정 시술 유형' in df.columns:
            for treatment in all_treatments:
                df[treatment] = df['특정 시술 유형'].str.replace(" ", "").apply(lambda x: 1 if treatment in x else 0)

    # 기존 '특정 시술 유형' 컬럼 삭제
    train_df.drop(columns=['특정 시술 유형'], inplace=True, errors='ignore')
    test_df.drop(columns=['특정 시술 유형'], inplace=True, errors='ignore')

    # 추가된 칼럼 목록 출력
    print(f"추가된 시술 유형 칼럼들: {list(all_treatments)}")
    return train_df, test_df

# 사용 예시
try:
    train, test = encode_treatment_types(train, test)
except:
    train, test = train, test

추가된 시술 유형 칼럼들: ['IUI', 'IVF', 'ICI', 'BLASTOCYST', 'GenericDI', 'AH', 'Unknown', 'ICSI', 'IVI']


### Ordinal Encoding
- 나이 변수 : '시술 당시 나이', '난자 기증자 나이', '정자 기증자 나이'
- 횟수 변수 : '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 
    'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수'

In [397]:
def ordinal_encoding(train_df, test_df):
    """
    나이 관련 변수를 순서형 숫자로 변환하고, 빈도 관련 변수를 정수 변환하는 함수.

    Parameters:
    - train_df: 학습 데이터 (DataFrame)
    - test_df: 테스트 데이터 (DataFrame)

    Returns:
    - train_df: 변환된 학습 데이터 (DataFrame)
    - test_df: 변환된 테스트 데이터 (DataFrame)
    """

    # 데이터 복사
    train_df = train_df.copy()
    test_df = test_df.copy()

    # 나이 관련 변수 리스트
    age_columns = ['시술 당시 나이', '난자 기증자 나이', '정자 기증자 나이']

    # 나이 변환 (존재하는 컬럼만 처리)
    for col in age_columns:
        if col in train_df.columns:
            unique_ages = train_df[col].dropna().unique()
            cleaned_ages = []

            for age in unique_ages:
                age_str = str(age).strip().replace("만", "").replace("세", "").replace(" ", "")

                # 나이 범위 처리
                if "-" in age_str:
                    age_str = age_str.split("-")[0]  # 범위 값이면 첫 번째 값 사용
                elif "이하" in age_str:
                    age_str = age_str.replace("이하", "")  # "이하" 제거

                # 숫자로 변환 가능한 경우 리스트에 추가
                if age_str.isdigit():
                    cleaned_ages.append(int(age_str))

            # 숫자로 변환하여 정렬
            sorted_ages = sorted(set(cleaned_ages))

            # 1부터 순차적으로 매핑
            age_mapping = {str(age): idx + 1 for idx, age in enumerate(sorted_ages)}

            # 매핑 적용
            train_df[col] = train_df[col].astype(str).str.replace("만", "").str.replace("세", "").str.split("-").str[0].str.replace("이하", "").map(age_mapping)
            test_df[col] = test_df[col].astype(str).str.replace("만", "").str.replace("세", "").str.split("-").str[0].str.replace("이하", "").map(age_mapping)

            # 결측값 0으로 대체
            train_df[col] = train_df[col].fillna(0)
            test_df[col] = test_df[col].fillna(0)

    # 빈도 관련 컬럼 변환 (존재하는 컬럼만 처리)
    freq_columns = ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', 
                    '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']

    for col in freq_columns:
        if col in train_df.columns:
            train_df[col] = train_df[col].apply(lambda x: int(str(x)[:1]) if pd.notna(x) and str(x)[0].isdigit() else x)
            test_df[col] = test_df[col].apply(lambda x: int(str(x)[:1]) if pd.notna(x) and str(x)[0].isdigit() else x)

    return train_df, test_df

# 사용 예시
train, test = ordinal_encoding(train, test)


### 배아 생성 주요 이유

In [398]:
def process_embryo_reason(train_df, test_df):
    # 데이터 복사
    train_df = train_df.copy()
    test_df = test_df.copy()

    # 새로운 컬럼 리스트
    new_columns = ['현재 시술용', '난자 저장용', '배아 저장용', '기증용', '연구용']

    # 각 컬럼을 0으로 초기화
    if '배아 생성 주요 이유' in train_df.columns:
        for col in new_columns:
            train_df[col] = 0
            test_df[col] = 0


        for index, value in train_df['배아 생성 주요 이유'].items():
            if pd.notna(value):  # 결측값이 아닌 경우만 처리
                for col in new_columns:
                    if col in str(value):  # 문자열 변환 후 확인
                        train_df.at[index, col] = 1


        for index, value in test_df['배아 생성 주요 이유'].items():
            if pd.notna(value):  # 결측값이 아닌 경우만 처리
                for col in new_columns:
                    if col in str(value):  # 문자열 변환 후 확인
                        test_df.at[index, col] = 1

    # 기존 '배아 생성 주요 이유' 컬럼 삭제
    train_df.drop(columns=['배아 생성 주요 이유'], inplace=True, errors='ignore')
    test_df.drop(columns=['배아 생성 주요 이유'], inplace=True, errors='ignore')

    return train_df, test_df

# 사용 예시
train, test = process_embryo_reason(train, test)

### MinMax Scaling
- "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수"

In [399]:
def minmax_scale_columns(train_df, test_df):
    """
    특정 변수들 중 실제 데이터에 존재하는 변수들에 대해 MinMax Scaling 적용.

    Parameters:
    - train_df: 학습 데이터 (DataFrame)
    - test_df: 테스트 데이터 (DataFrame)

    Returns:
    - train_df: MinMax Scaling 적용된 학습 데이터 (DataFrame)
    - test_df: MinMax Scaling 적용된 테스트 데이터 (DataFrame)
    """

    # MinMax Scaling 대상 변수 리스트
    scale_columns = [
        "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수", "저장된 배아 수", "미세주입 후 저장된 배아 수",
        "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수",
        "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수"
    ]

    # 실제 데이터에 존재하는 컬럼 필터링
    existing_columns = [col for col in scale_columns if col in train_df.columns]

    if not existing_columns:
        print("MinMax Scaling 적용할 컬럼이 없습니다.")
        return train_df, test_df

    # MinMaxScaler 초기화
    scaler = MinMaxScaler()

    # Train 데이터 스케일링
    train_df[existing_columns] = scaler.fit_transform(train_df[existing_columns])

    # Test 데이터 동일 스케일링 적용
    test_df[existing_columns] = scaler.transform(test_df[existing_columns])

    # 적용된 컬럼 출력
    print(f"MinMax Scaling 적용된 컬럼: {existing_columns}")

    return train_df, test_df

# 사용 예시
train, test = minmax_scale_columns(train, test)

MinMax Scaling 적용된 컬럼: ['총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수']


### 이상치 제거

In [400]:
train.shape, test.shape

((256351, 78), (90067, 77))

In [401]:
def remove_outliers_from_train(train_df, test_df):
    """
    지정된 컬럼에서 이상치를 제거하는 함수 (IQR 방식 사용, 학습 데이터만 변경)

    Parameters:
    - train_df: 학습 데이터 (DataFrame)
    - test_df: 테스트 데이터 (DataFrame) (변경되지 않음)

    Returns:
    - train_df: 이상치 제거 후 학습 데이터 (DataFrame)
    - test_df: 원본 유지된 테스트 데이터 (DataFrame)
    """

    # 데이터 복사 (train만 변경)
    train_df = train_df.copy()
    
    scale_columns = [
        "임신 시도 또는 마지막 임신 경과 연수", "총 생성 배아 수", "미세주입된 난자 수",
        "미세주입에서 생성된 배아 수", "저장된 배아 수", "미세주입 후 저장된 배아 수",
        "해동된 배아 수", "해동 난자 수", "수집된 신선 난자 수", "저장된 신선 난자 수",
        "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수"
    ]

    # 이상치가 제거된 행 개수를 저장할 딕셔너리
    removed_counts = {}

    for col in scale_columns:
        if col in train_df.columns:
            # IQR 계산 (10~90 분위수 기준)
            Q1 = train_df[col].quantile(0)
            Q3 = train_df[col].quantile(0.99)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # 이상치 탐지
            outliers = (train_df[col] < lower_bound) | (train_df[col] > upper_bound)

            # 삭제할 행 개수 저장
            removed_counts[col] = outliers.sum()

            # 이상치 제거
            train_df = train_df[~outliers]

    # 이상치 제거된 개수 출력
    print(f"이상치가 제거된 컬럼 및 행 개수: {removed_counts}")

    return train_df, test_df  # test_df는 변경되지 않음

# 사용 예시
train, test = remove_outliers_from_train(train, test)

이상치가 제거된 컬럼 및 행 개수: {'총 생성 배아 수': 2, '미세주입된 난자 수': 0, '미세주입에서 생성된 배아 수': 0, '저장된 배아 수': 29, '미세주입 후 저장된 배아 수': 87, '해동된 배아 수': 74, '해동 난자 수': 1445, '수집된 신선 난자 수': 0, '저장된 신선 난자 수': 2329, '혼합된 난자 수': 0, '파트너 정자와 혼합된 난자 수': 0, '기증자 정자와 혼합된 난자 수': 30}


In [402]:
train.shape, test.shape

((252355, 78), (90067, 77))

### [도은_논문기반] 난자가 없는데 임신이 된 경우 이상치로 처리

In [ ]:
print(train.shape)
train_noegg = train.loc[(((train['해동 난자 수'] + train['수집된 신선 난자 수']) == 0) & (train['임신 성공 여부'] == 1))]
train = train.loc[~(((train['해동 난자 수'] + train['수집된 신선 난자 수']) == 0) & (train['임신 성공 여부'] == 1))]
print(train.shape)

(238928, 78)
(238928, 78)


In [403]:
# noegg2
print(train.shape)
train_noegg = train.loc[(((train['해동 난자 수'] + train['수집된 신선 난자 수']) == 0))]
train = train.loc[~(((train['해동 난자 수'] + train['수집된 신선 난자 수']) == 0))]
print(train.shape)

print(test.shape)
test_noegg = test.loc[((test['해동 난자 수'] + test['수집된 신선 난자 수']) == 0)]
test = test.loc[~((test['해동 난자 수'] + test['수집된 신선 난자 수']) == 0)]
print(test.shape)

(252355, 78)
(193657, 78)
(90067, 77)
(69665, 77)


### [도은] 배아가 없는 사람 따로 모델링

In [372]:
print(train.shape)
train_noembryo = train.loc[(((train['총 생성 배아 수']) == 0) )]
train =  train.loc[~(((train['총 생성 배아 수']) == 0) )]
print(train.shape)

print(test.shape)
test_noembryo = test.loc[(((test['총 생성 배아 수']) == 0) )]
test =  test.loc[~(((test['총 생성 배아 수']) == 0) )]
print(test.shape)

(203858, 68)
(193151, 68)
(71841, 67)
(68087, 67)


### 원 핫 인코딩

In [404]:
def one_hot_encode_columns(train_df, test_df):
    """
    train_df의 object 데이터 타입인 변수들을 원핫인코딩하여 변환하고,
    test_df는 train_df에서 생성된 변수에 맞게 변환.

    Parameters:
    - train_df: 학습 데이터 (DataFrame)
    - tesegg테스트 데이터 (DataFrame)

    Returns:
    - train_df: 원핫인코딩 적용된 학습 데이터 (DataFrame)
    - test_df: 원핫인코딩 적용된 테스트 데이터 (DataFrame)
    """

    # 타겟 변수를 제외한 원핫인코딩 대상 변수 리스트 (object 타입인 컬럼)
    categorical_columns = [col for col in train_df.select_dtypes(include=['object']).columns.tolist()
                           if col != "임신 성공 여부"]

    if not categorical_columns:
        print("원핫인코딩 적용할 컬럼이 없습니다.")
        return train_df, test_df

    # OneHotEncoder 초기화 (sparse_output=True로 설정하여 최적화)
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

    # Train 데이터 인코딩
    train_encoded = encoder.fit_transform(train_df[categorical_columns])
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(), index=train_df.index)

    # Test 데이터 인코딩 (Train에서 학습한 변수만 적용)
    test_encoded = encoder.transform(test_df[categorical_columns])
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(), index=test_df.index)

    # 기존 object 타입 컬럼 삭제 후 인코딩된 컬럼 추가 (reset_index 제거)
    train_df = train_df.drop(columns=categorical_columns)
    test_df = test_df.drop(columns=categorical_columns)

    train_df = pd.concat([train_df, train_encoded_df], axis=1)
    test_df = pd.concat([test_df, test_encoded_df], axis=1)

    # 적용된 컬럼 출력
    print(f"원핫인코딩 적용된 컬럼 개수: {len(train_encoded_df.columns)}")
    print(f"원핫인코딩 적용된 컬럼 리스트: {train_encoded_df.columns.tolist()}")

    return train_df, test_df

# 사용 예시
train, test = one_hot_encode_columns(train, test)
train_noegg, test_noegg = one_hot_encode_columns(train_noegg, test_noegg)

원핫인코딩 적용된 컬럼 개수: 14
원핫인코딩 적용된 컬럼 리스트: ['시술 시기 코드_TRCMWS', '시술 시기 코드_TRDQAZ', '시술 시기 코드_TRJXFG', '시술 시기 코드_TRVNRY', '시술 시기 코드_TRXQMD', '시술 시기 코드_TRYBLT', '시술 시기 코드_TRZKPL', '시술 유형_IVF', '난자 출처_기증 제공', '난자 출처_본인 제공', '정자 출처_기증 제공', '정자 출처_미할당', '정자 출처_배우자 및 기증 제공', '정자 출처_배우자 제공']
원핫인코딩 적용된 컬럼 개수: 15
원핫인코딩 적용된 컬럼 리스트: ['시술 시기 코드_TRCMWS', '시술 시기 코드_TRDQAZ', '시술 시기 코드_TRJXFG', '시술 시기 코드_TRVNRY', '시술 시기 코드_TRXQMD', '시술 시기 코드_TRYBLT', '시술 시기 코드_TRZKPL', '시술 유형_DI', '시술 유형_IVF', '난자 출처_기증 제공', '난자 출처_본인 제공', '난자 출처_알 수 없음', '정자 출처_기증 제공', '정자 출처_미할당', '정자 출처_배우자 제공']


### 분산 0인 칼람 삭제

In [405]:
def remove_single_value_columns(train_df, test_df, fill_value=999):
    """
    NaN을 임시 값으로 채운 후, 유일한 값이 하나뿐인 컬럼을 삭제하는 함수

    Parameters:
    - train_df: 학습 데이터 (DataFrame)
    - test_df: 테스트 데이터 (DataFrame)
    - fill_value: NaN을 대체할 임시 값 (기본값: 999)

    Retgg
    - train_df: 단일 값 컬럼 삭제 후 학습 데이터 (DataFrame)
    - test_df: 단일 값 컬럼 삭제 후 테스트 데이터 (DataFrame)
    """

    # NaN을 임시 값으로 채운 후, 유일한 값이 하나뿐인 컬럼 찾기
    single_value_cols_train = train_df.fillna(fill_value).nunique() == 1

    # 제거할 컬럼 리스트
    cols_to_drop = train_df.columns[single_value_cols_train].tolist()

    # 데이터에서 해당 컬럼 삭제
    train_df = train_df.drop(columns=cols_to_drop)
    test_df = test_df.drop(columns=cols_to_drop)

    # 삭제된 컬럼 출력
    print(f"삭제된 단일 값 컬럼: {cols_to_drop}")

    return train_df, test_df


train, test = remove_single_value_columns(train, test)
train_noegg, test_noegg = remove_single_value_columns(train_noegg, test_noegg)


삭제된 단일 값 컬럼: ['불임 원인 - 여성 요인', '해동 난자 수', '기증 배아 사용 여부', '난자 해동 경과일', 'IUI', 'ICI', 'GenericDI', 'IVI', '연구용', '시술 유형_IVF']
삭제된 단일 값 컬럼: ['불임 원인 - 여성 요인', '불임 원인 - 정자 면역학적 요인', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '난자 해동 경과일', '난자 혼합 경과일']


# 저장

In [406]:
train.to_csv("./datasets/train_Preprocessed_noegg2.csv", index=True)
test.to_csv("./datasets/test_Preprocessed_noegg2.csv", index=True)

In [407]:
train_noegg.to_csv("./datasets/train_Preprocessed_noegg_subset.csv", index=True)
test_noegg.to_csv("./datasets/test_Preprocessed_noegg_subset.csv", index=True)